Hej hej test

In [2]:
%pip install --index-url https://test.pypi.org/simple/ --no-deps gc2-python-client
import gc2
from pandas import DataFrame, concat
import tabulate
import json
import folium



Looking in indexes: https://test.pypi.org/simple/
You should consider upgrading via the '/home/mh/Source/gc2-python-client/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
local = gc2.Gc2("https://dk.gc2.io")
# local = gc2.Gc2("http://127.0.0.1:8080")
local.set_authentication(user="dk_test", pw="pw1234!", db="dk")
# local.set_authentication(user="mydb", pw="hawk2000", db="mydb")


Exception: Error 400: {"error":"invalid_grant","error_description":"Could not authenticate the user. Check username and password","code":400,"_execution_time":0.011}

In [4]:
query = gc2.Sql(local)
query.run("select ejerlavsnavn, matrikelnummer, the_geom from matrikel.jordstykke limit 10")
# query.table
tabulate.tabulate(query.table,  tablefmt='html')


ejerlavsnavn,matrikelnummer,the_geom
"Tåstrup, Holbæk Jorder",4ep,
"Tåstrup, Holbæk Jorder",4ei,
"Tåstrup, Holbæk Jorder",4iq,
"Tåstrup, Holbæk Jorder",2ae,
"Tåstrup, Holbæk Jorder",2ai,
"Tåstrup, Holbæk Jorder",8,
"Tåstrup, Holbæk Jorder",7000æ,
"Tåstrup, Holbæk Jorder",17,
"Tåstrup, Holbæk Jorder",3ei,
"Tåstrup, Holbæk Jorder",3em,


In [ ]:
m = folium.Map(
    location=[56.1759, 10.6016],
    tiles="cartodbpositron",
    zoom_start=7,
)
folium.GeoJson(query.geojson, name="geojson").add_to(m)
folium.LayerControl().add_to(m)
m